In [1]:
import numpy as np
#from soliket.ccl import CCL
from cobaya.model import get_model
from cobaya.likelihood import Likelihood
import pyccl as ccl

In [2]:
!pwd

/Users/Pablo/Code/SOLikeT/soliket/tests


In [3]:
class Tester(Likelihood):
    # Cross and auto data

    auto_file: str = '/Users/Pablo/Code/SOLikeT/input/clgg.txt'
    cross_file: str = '/Users/Pablo/Code/SOLikeT/input/clkg.txt'
    dndz_file: str = '/Users/Pablo/Code/SOLikeT/input/dndz.txt'

    params = {'b1': 1, 's1': 1} 

    #ell_file: str = "/Users/Pablo/Code/SOLikeT/soliket/data/simulated_ccl/ell.npy"
    #cl_file: str = "/Users/Pablo/Code/SOLikeT/soliket/data/simulated_ccl/cls.npy"
    #dcl_file: str = "/Users/Pablo/Code/SOLikeT/soliket/data/simulated_ccl/dcls.npy"

    def initialize(self):
        #self.cl_data = np.load(self.cl_file)
        #self.dcl_data = np.load(self.dcl_file)
        #self.ell_data = np.load(self.ell_file)
        data_auto = np.loadtxt(self.auto_file)
        data_cross = np.loadtxt(self.cross_file)
        self.dndz = np.loadtxt(self.dndz_file)

        # Get data
        self.ell_auto = data_auto[0]
        self.cl_auto = data_auto[1]
        self.cl_auto_err = data_auto[2]

        self.ell_cross = data_cross[0]
        self.cl_cross = data_cross[1]
        self.cl_cross_err = data_cross[2]   

    def get_requirements(self):
        return {'CCL': {#"methods": {'theory': self._get_theory},
                        "kmax": 10,
                        "nonlinear": True}}

    def logp(self, **pars):
        cosmo = self.provider.get_CCL()['cosmo']
        #cosmo = results['theory']

        tracer_g = ccl.NumberCountsTracer(cosmo, has_rsd=False, dndz = self.dndz.T, 
                                            bias =(self.dndz[:,0], pars['b1']*np.ones(len(self.dndz[:,0]))), 
                                            mag_bias = (self.dndz[:,0], pars['s1']*np.ones(len(self.dndz[:,0])))
                                            )
        tracer_k = ccl.CMBLensingTracer(cosmo, z_source = 1060)

        cl_gg = ccl.cls.angular_cl(cosmo, tracer_g, tracer_g, self.ell_auto)
        cl_kg = ccl.cls.angular_cl(cosmo, tracer_k, tracer_g, self.ell_cross)        
        
        #cl_gg, cl_kg = results['theory']
        delta = np.concatenate([cl_gg - self.cl_auto, cl_kg - self.cl_cross])
        sigma = np.concatenate([self.cl_auto_err, self.cl_cross_err])
        chi2 = delta**2/sigma**2.
        #chi2 = np.dot(r, self.invcov.dot(r))
        return -0.5 * np.sum(chi2)

In [4]:
cosmo_params = {
    "Omega_c": 0.25,
    "Omega_b": 0.05,
    "h": 0.67,
    "n_s": 0.96
}

info = {"params": {#"omch2": cosmo_params['Omega_c'] * cosmo_params['h'] ** 2.,
                   "ombh2": 0.024499999999999997, #cosmo_params['Omega_b'] * cosmo_params['h'] ** 2.,
                   "H0": 70, #cosmo_params['h'] * 100,
                   "ns": 0.965, #cosmo_params['n_s'],
                   #"As": 2e-9,
                   #"logA": {'prior':{'min': 1.61, 'max': 3.91}, 'latex': '\log(10^{10} A_\mathrm{s})', 'drop': True},
                   "logA": {'value': 3, 'latex': '\log(10^{10} A_\mathrm{s})', 'drop': True},
                   "As": {'value':lambda logA: 1e-10*np.exp(logA), 'latex': 'A_\mathrm{s}'},
                   #"omegab": {'value':0.04914741792, 'latex': '\Omega_\mathrm{b}', 'drop': True},
                   #"omegam": {'prior':{'min': 0.1, 'max': 0.9}, 'latex': '\Omega_\mathrm{m}', 'drop': True},
                   "omegam": {'value': 0.3, 'latex': '\Omega_\mathrm{m}', 'drop': True},
                   #"omch2": {'value':lambda omegam, omegab, mnu, H0: (omegam-omegab)*(H0/100)**2-(mnu*(3.046/3)**0.75)/94.0708, 'latex': '\Omega_\mathrm{c} h^2'},
                    "omch2": {'value':lambda omegam, ombh2, mnu, H0: (omegam*(H0/100)**2-ombh2)-(mnu*(3.046/3)**0.75)/94.0708, 'latex': '\Omega_\mathrm{c} h^2'},
                   "tau": 0.05,
                   "mnu": 0.0, 
                   "b1": {'prior':{'min': 0.99, 'max': 1.01}, 'latex': 'b_1'},
                   "s1": 0.4},
        #"likelihood": {"Tester": Tester},
        "likelihood": {"soliket.ccl.Tester": {"python_path": "/Users/Pablo/Code/SOLikeT/", 
                                              "auto_file": '/Users/Pablo/Code/SOLikeT/input/clgg.txt', 
                                              "cross_file": '/Users/Pablo/Code/SOLikeT/input/clkg.txt',
                                              "dndz_file": '/Users/Pablo/Code/SOLikeT/input/dndz.txt',
                                             }},
        "theory": {
            "camb": None,
            "soliket.ccl.CCL": {"python_path": "/Users/Pablo/Code/SOLikeT/", "nonlinear": True}, #{"external": CCL, "nonlinear": True}
        },
        "debug": True, "stop_at_error": True, 
       "sampler": {"evaluate": None} #{"mcmc": {'max_tries': 1000}}
       }

In [5]:
from cobaya.run import run

updated_info, sampler = run(info)

 2021-03-18 12:57:51,929 [outputdummy] No output requested. Doing nothing.
 2021-03-18 12:57:52,748 [run] Input info updated with defaults (dumped to YAML):
theory:
  camb:
    version: null
    path: null
    speed: 0.3
    stop_at_error: false
    extra_args: null
    use_renames: false
    external_primordial_pk: false
    renames:
      omegabh2: ombh2
      omegach2: omch2
      omegal: omega_de
      omegak: omk
      yhe: YHe
      yheused: YHe
      YpBBN: Y_p
      zrei: zre
  soliket.ccl.CCL:
    kmax: 0
    nonlinear: true
    z: []
    extra_args: {}
    logz:
    - -3.0
    - -2.9594537403680654
    - -2.9189074807361313
    - -2.8783612211041967
    - -2.8378149614722625
    - -2.797268701840328
    - -2.7567224422083934
    - -2.716176182576459
    - -2.6756299229445246
    - -2.6350836633125905
    - -2.594537403680656
    - -2.5539911440487217
    - -2.513444884416787
    - -2.472898624784853
    - -2.4323523651529184
    - -2.391806105520984
    - -2.3512598458890497


 2021-03-18 12:57:52,803 [camb.transfers] Computing new state
 2021-03-18 12:57:52,804 [camb] Setting parameters: {'ombh2': 0.024499999999999997, 'H0': 70, 'omch2': 0.12249999999999997, 'tau': 0.05, 'mnu': 0.0} and {'kmax': 10, 'redshifts': array([1.10000000e+03, 1.00195087e+03, 9.12641400e+02, 8.31292584e+02,
       7.57194842e+02, 6.89701844e+02, 6.28224873e+02, 5.72227688e+02,
       5.21221844e+02, 4.74762435e+02, 4.32444212e+02, 3.93898049e+02,
       3.58787719e+02, 3.26806970e+02, 2.97676842e+02, 2.71143246e+02,
       2.46974737e+02, 2.24960501e+02, 2.04908518e+02, 1.86643879e+02,
       1.70007270e+02, 1.54853574e+02, 1.41050611e+02, 1.28477984e+02,
       1.17026025e+02, 1.06594843e+02, 9.70934504e+01, 8.84389698e+01,
       8.05559114e+01, 7.33755139e+01, 6.68351452e+01, 6.08777561e+01,
       5.54513823e+01, 5.05086915e+01, 4.60065702e+01, 4.19057481e+01,
       3.81704551e+01, 3.47681097e+01, 3.16690342e+01, 2.88461966e+01,
       2.62749743e+01, 2.39329394e+01, 2.17996630

In [5]:
from cobaya.run import run

updated_info, sampler = run(info)

 2021-03-18 12:27:20,789 [outputdummy] No output requested. Doing nothing.
 2021-03-18 12:27:20,818 [run] Input info updated with defaults (dumped to YAML):
theory:
  camb:
    version: null
    path: null
    speed: 0.3
    stop_at_error: false
    extra_args: null
    use_renames: false
    external_primordial_pk: false
    renames:
      omegabh2: ombh2
      omegach2: omch2
      omegal: omega_de
      omegak: omk
      yhe: YHe
      yheused: YHe
      YpBBN: Y_p
      zrei: zre
  soliket.ccl.CCL:
    kmax: 0
    nonlinear: true
    z: []
    extra_args: {}
    logz:
    - -3.0
    - -2.9389758314630483
    - -2.8779516629260966
    - -2.8169274943891445
    - -2.7559033258521928
    - -2.694879157315241
    - -2.6338549887782894
    - -2.5728308202413377
    - -2.511806651704386
    - -2.4507824831674343
    - -2.389758314630482
    - -2.3287341460935305
    - -2.2677099775565788
    - -2.206685809019627
    - -2.1456616404826754
    - -2.0846374719457232
    - -2.023613303408771

 2021-03-18 12:27:20,894 [camb] Setting attributes of CAMBparams: {'Want_CMB': False, 'Want_cl_2D_array': False, 'WantCls': False, 'NonLinear': 'NonLinear_pk'}
 2021-03-18 12:27:21,112 [camb] Got parameters {'ns': 0.965, 'As': 2.008553692318767e-09}
 2021-03-18 12:27:21,113 [camb] Computing new state
 2021-03-18 12:27:21,710 [soliket.ccl.ccl] Got parameters {}
 2021-03-18 12:27:21,711 [soliket.ccl.ccl] Computing new state
 2021-03-18 12:27:21,721 [tester] Got parameters {'b1': 1.0004447762914368, 's1': 0.4}
 2021-03-18 12:27:21,722 [tester] Computing new state
 2021-03-18 12:27:21,731 [tester] Computed log-likelihood = -41.6874
 2021-03-18 12:27:21,731 [model] Computed derived parameters: {'As': 2.008553692318767e-09, 'omch2': 0.12249999999999997}
 2021-03-18 12:27:21,732 [evaluate] log-posterior  = -37.7754
 2021-03-18 12:27:21,733 [evaluate] log-prior      = 3.91202
 2021-03-18 12:27:21,734 [evaluate]    logprior_0 = 3.91202
 2021-03-18 12:27:21,734 [evaluate] log-likelihood = -41.68